In [ ]:
#Uncomment if you're using colab
#Setting up google drive 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks')

In [ ]:
import my_utils as mu
import torch
from torch import nn
from IPython import display

# Concise Implementation of Softmax Regression

* Goal: use high-level APIs of PyTorch for implementing Softmax Regression for classification. 

In [ ]:
batch_size = 256
train_iter, test_iter = mu.load_data_fashion_mnist(batch_size)
# type(train_iter)

In [ ]:
X, y = next(iter(train_iter)) # first batch
print(X.size())
print(y)

# Defining the Model and Initialization

* Each example is represented by a fixed-length vector: we flatten each $28 \times 28$ image, treating it as vector of length 784.

* Because our dataset has 10 classes, our network will have an output dimension of 10.
* So, our weights `W` will be a $784 \times 10$ matrix and the biases `b` will constitute a $10 \times 1$ row vector.
* We initialize `W` using a Gaussian distribution and `b` with 0.
* Softmax regression can be implemented as a Fully-Connected (i.e Linear) layer.


In [ ]:
class Net(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(Net, self).__init__()
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        self.Linear1 = nn.Linear(num_inputs, num_outputs)
        torch.nn.init.normal_(self.Linear1.weight, std=0.01) #init the weights
        torch.nn.init.zeros_(self.Linear1.bias) #init the bias
        
    def forward(self, x):
        x = x.view(-1, self.num_inputs)
        out = self.Linear1(x)
        return out

num_inputs, num_outputs = 784, 10
net = Net(num_inputs, num_outputs)
print(net)

# Alternative Initialization
* This is useful if you have multiple layers of the same type and you want them to be initialized in the same way.

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear): # by checking type we can init different layers in different ways
        torch.nn.init.normal_(m.weight, std=0.01)
        torch.nn.init.zeros_(m.bias)

net.apply(init_weights);
# print(net)

# Loss Function

* Use PyTorch's implementation of Softmax-Cross Entropy loss to avoid numerical instabilities.
    * The input to loss function are the logits logits $\mathbf{o}$ (and not softmax outputs)

In [ ]:
loss = nn.CrossEntropyLoss()

# Optimization Algorithm


* Minibatch SGD with a learning rate of 0.1 as the optimization algorithm.


In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

# Evaluation


In [ ]:
def accuracy(y_hat, y):  #y_hat is a matrix; 2nd dimension stores prediction scores for each class.
    """Compute the number of correct predictions."""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1) # Predicted class is the index of max score         
    cmp = (y_hat.type(y.dtype) == y)  # because`==` is sensitive to data types
    return float(torch.sum(cmp)) # Taking the sum yields the number of correct predictions.

# Example: only 1 sample is correctly classified.
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
accuracy(y_hat, y) / len(y)

0.5

In [ ]:
class Accumulator:  
    """For accumulating sums over `n` variables."""
    def __init__(self, n):
        self.data = [0.0] * n # [0, 0, ..., 0]
    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]
    def reset(self):
        self.data = [0.0] * len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
def evaluate_accuracy(net, data_iter): 
    """Compute the accuracy for a model on a dataset."""
    metric = Accumulator(2)  # No. of correct predictions, no. of predictions
    for _, (X, y) in enumerate(data_iter):
        metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

* The accuracy of the model prior to training should be close to random guessing, i.e., 0.1 for 10 classes.


In [ ]:
evaluate_accuracy(net, test_iter)

# Training

* The training loop for softmax regression looks strikingly familiar with that of linear regression. 
* Here we refactor the implementation to make it reusable.
    * First, we define a function to train for one epoch.

In [ ]:
def train_epoch_ch3(net, train_iter, loss, optimizer):  
    """The training function for one epoch."""
    # Set the model to training mode
    if isinstance(net, torch.nn.Module):
        net.train()
    # Sum of training loss, sum of training accuracy, no. of examples
    metric = Accumulator(3)
    for X, y in train_iter:
        # Compute gradients and update parameters
        y_hat = net(X)
        l = loss(y_hat, y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        metric.add(float(l) * len(y), accuracy(y_hat, y), y.size().numel())
    # Return training loss and training accuracy
    return metric[0] / metric[2], metric[1] / metric[2]

# Training

* The following class will be used to plot training and validation accuracy as well as loss evolution over training loop. 

In [ ]:
class Animator:  
    """For plotting data in animation."""
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(3.5, 2.5)):
        # Incrementally plot multiple lines
        if legend is None:
            legend = []
        mu.use_svg_display()
        self.fig, self.axes = mu.plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1:
            self.axes = [self.axes, ]
        # Use a lambda function to capture arguments
        self.config_axes = lambda: mu.set_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts

    def add(self, x, y):
        # Add multiple data points into the figure
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        display.display(self.fig)
        display.clear_output(wait=True)

# Training

* The following function trains the model (`net`) on a training set (`train_iter`) for `num_epochs`.
* At the end of each epoch, the model is evaluated on a testing set (`test_iter`).
* `Animator` for visualizing the training progress.

In [ ]:
def train_ch3(net, train_iter, test_iter, loss, num_epochs, optimizer):  
    """Train a model."""
    animator = Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0.3, 0.9],
                        legend=['train loss', 'train acc', 'test acc'])
    for epoch in range(num_epochs):
        train_metrics = train_epoch_ch3(net, train_iter, loss, optimizer)
        test_acc = evaluate_accuracy(net, test_iter)
        animator.add(epoch + 1, train_metrics + (test_acc,))
    train_loss, train_acc = train_metrics
    assert train_loss < 0.5, train_loss
    assert train_acc <= 1 and train_acc > 0.7, train_acc
    assert test_acc <= 1 and test_acc > 0.7, test_acc

In [ ]:
num_epochs = 30
train_ch3(net, train_iter, test_iter, loss, num_epochs, optimizer)

# Summary

* Using PyTorch's high-level APIs, we can implement softmax regression much more concisely.
* From a computational perspective, implementing softmax regression has intricacies. 
* Note that in many cases, PyTorch takes additional precautions beyond these most well-known tricks to ensure numerical stability